In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime

In [2]:
from os import walk

filenames = next(walk('../data/'), (None, None, []))[2]  # [] if no file
print(filenames)

['entry_loc3_w_filter.csv', 'states.csv', 'census.csv', 'entry_loc2.csv', 'CBSA.csv', 'entry_loc.csv', 'entry.csv', 'distr.csv', 'warehouse_loc.csv']


In [3]:
# Load data files
entry2_data = pd.read_csv("../data/entry_loc2.csv")
CBSA = pd.read_csv("../data/CBSA.csv")

In [4]:
# Clean up CDP city name2s
lst = [' city',' village',' CDP',' town',' borough',' city and borough', ' municipality', ' urban county']
entry2_data['city']= entry2_data['city'].str.strip()
entry2_data['city'] = entry2_data['city'].replace(lst,'', regex=True)
entry2_data['city']= entry2_data['city'].str.strip()
"""entry2_data['citys'] = entry2_data['city'] +',' #note the space character
entry2_data['city_'] = entry2_data['city'] +'-'"""
entry2_data['city']

0             Aullville
1                 Gibbs
2             West Line
3               De Witt
4           Arrow Point
              ...      
17308    South Portland
17309            Bangor
17310          Lewiston
17311          Portland
17312        Washington
Name: city, Length: 17313, dtype: object

In [5]:
# Drop irrelevant columns - CBSA
CBSA.drop(CBSA.columns.difference(['NAME']), 1, inplace=True)
CBSA

/tmp/ipykernel_137658/3748014381.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  CBSA.drop(CBSA.columns.difference(['NAME']), 1, inplace=True)


,NAME
0,"Iowa City, IA Metro Area"
1,"Hobbs, NM Micro Area"
2,"Holland, MI Micro Area"
3,"Homosassa Springs, FL Metro Area"
4,"Hood River, OR Micro Area"
...,...
940,"Hilton Head Island-Bluffton-Beaufort, SC Metro..."
941,"Hinesville, GA Metro Area"
942,"Guayama, PR Metro Area"
943,"Indianola, MS Micro Area"


In [6]:
# Drop irrelevant columns - CDP
entry2_data.drop(entry2_data.columns.difference(['city','STUSAB']), 1, inplace=True)
entry2_data

/tmp/ipykernel_137658/374498205.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  entry2_data.drop(entry2_data.columns.difference(['city','STUSAB']), 1, inplace=True)


,city,STUSAB
0,Aullville,MO
1,Gibbs,MO
2,West Line,MO
3,De Witt,MO
4,Arrow Point,MO
...,...,...
17308,South Portland,ME
17309,Bangor,ME
17310,Lewiston,ME
17311,Portland,ME


In [7]:
# Cross join the entry data with CBSAs
merged = entry2_data.merge(CBSA, how='cross')
merged

,city,STUSAB,NAME
0,Aullville,MO,"Iowa City, IA Metro Area"
1,Aullville,MO,"Hobbs, NM Micro Area"
2,Aullville,MO,"Holland, MI Micro Area"
3,Aullville,MO,"Homosassa Springs, FL Metro Area"
4,Aullville,MO,"Hood River, OR Micro Area"
...,...,...,...
16360780,Washington,DC,"Hilton Head Island-Bluffton-Beaufort, SC Metro..."
16360781,Washington,DC,"Hinesville, GA Metro Area"
16360782,Washington,DC,"Guayama, PR Metro Area"
16360783,Washington,DC,"Indianola, MS Micro Area"


In [8]:
# CDP name is in the CBSA name
merged['contains'] = merged.apply(lambda x: x.city in x.NAME, axis=1)
#merged['contains'] = merged.apply(lambda x: x.citys in x.NAME or x.city_ in x.NAME, axis=1)
merged

,city,STUSAB,NAME,contains
0,Aullville,MO,"Iowa City, IA Metro Area",False
1,Aullville,MO,"Hobbs, NM Micro Area",False
2,Aullville,MO,"Holland, MI Micro Area",False
3,Aullville,MO,"Homosassa Springs, FL Metro Area",False
4,Aullville,MO,"Hood River, OR Micro Area",False
...,...,...,...,...
16360780,Washington,DC,"Hilton Head Island-Bluffton-Beaufort, SC Metro...",False
16360781,Washington,DC,"Hinesville, GA Metro Area",False
16360782,Washington,DC,"Guayama, PR Metro Area",False
16360783,Washington,DC,"Indianola, MS Micro Area",False


In [9]:
merged = merged.loc[merged.contains]
merged

,city,STUSAB,NAME,contains
11821,Jacksonville,MO,"Jacksonville, FL Metro Area",True
11822,Jacksonville,MO,"Jacksonville, IL Micro Area",True
11823,Jacksonville,MO,"Jacksonville, NC Metro Area",True
11824,Jacksonville,MO,"Jacksonville, TX Micro Area",True
11869,Jacksonville,MO,"Anniston-Oxford-Jacksonville, AL Metro Area",True
...,...,...,...,...
16359051,Portland,ME,"Portland-Vancouver-Hillsboro, OR-WA Metro Area",True
16360205,Washington,DC,"Washington-Arlington-Alexandria, DC-VA-MD-WV M...",True
16360206,Washington,DC,"Washington Court House, OH Micro Area",True
16360500,Washington,DC,"Washington, IN Micro Area",True


In [10]:
# CDP state is in the CBSA name
merged['contains2'] = merged.apply(lambda x: x.STUSAB in x.NAME, axis=1)
merged

/tmp/ipykernel_137658/2895750497.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['contains2'] = merged.apply(lambda x: x.STUSAB in x.NAME, axis=1)


,city,STUSAB,NAME,contains,contains2
11821,Jacksonville,MO,"Jacksonville, FL Metro Area",True,False
11822,Jacksonville,MO,"Jacksonville, IL Micro Area",True,False
11823,Jacksonville,MO,"Jacksonville, NC Metro Area",True,False
11824,Jacksonville,MO,"Jacksonville, TX Micro Area",True,False
11869,Jacksonville,MO,"Anniston-Oxford-Jacksonville, AL Metro Area",True,False
...,...,...,...,...,...
16359051,Portland,ME,"Portland-Vancouver-Hillsboro, OR-WA Metro Area",True,False
16360205,Washington,DC,"Washington-Arlington-Alexandria, DC-VA-MD-WV M...",True,True
16360206,Washington,DC,"Washington Court House, OH Micro Area",True,False
16360500,Washington,DC,"Washington, IN Micro Area",True,False


In [11]:
merged = merged.loc[merged.contains2]
merged

,city,STUSAB,NAME,contains,contains2
324055,Madison,MO,"Fort Madison-Keokuk, IA-IL-MO Micro Area",True,True
534775,Fayette,MO,"Fayetteville-Springdale-Rogers, AR-MO Metro Area",True,True
654324,Kennett,MO,"Kennett, MO Micro Area",True,True
660233,Branson,MO,"Branson, MO Micro Area",True,True
660590,Mexico,MO,"Mexico, MO Micro Area",True,True
...,...,...,...,...,...
16356215,South Portland,ME,"Portland-South Portland, ME Metro Area",True,True
16357576,Bangor,ME,"Bangor, ME Metro Area",True,True
16358400,Lewiston,ME,"Lewiston-Auburn, ME Metro Area",True,True
16359050,Portland,ME,"Portland-South Portland, ME Metro Area",True,True


In [12]:
merged['lst'] = merged['NAME'].str.split('-|, ')
merged['checkisin'] = [c in l for c, l in zip(merged['city'], merged['lst'])]
merged = merged.loc[merged.checkisin]
merged

/tmp/ipykernel_137658/2227580837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['lst'] = merged['NAME'].str.split('-|, ')
/tmp/ipykernel_137658/2227580837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['checkisin'] = [c in l for c, l in zip(merged['city'], merged['lst'])]


,city,STUSAB,NAME,contains,contains2,lst,checkisin
654324,Kennett,MO,"Kennett, MO Micro Area",True,True,"[Kennett, MO Micro Area]",True
660233,Branson,MO,"Branson, MO Micro Area",True,True,"[Branson, MO Micro Area]",True
660590,Mexico,MO,"Mexico, MO Micro Area",True,True,"[Mexico, MO Micro Area]",True
665306,Maryville,MO,"Maryville, MO Micro Area",True,True,"[Maryville, MO Micro Area]",True
669737,West Plains,MO,"West Plains, MO Micro Area",True,True,"[West Plains, MO Micro Area]",True
...,...,...,...,...,...,...,...
16356215,South Portland,ME,"Portland-South Portland, ME Metro Area",True,True,"[Portland, South Portland, ME Metro Area]",True
16357576,Bangor,ME,"Bangor, ME Metro Area",True,True,"[Bangor, ME Metro Area]",True
16358400,Lewiston,ME,"Lewiston-Auburn, ME Metro Area",True,True,"[Lewiston, Auburn, ME Metro Area]",True
16359050,Portland,ME,"Portland-South Portland, ME Metro Area",True,True,"[Portland, South Portland, ME Metro Area]",True


In [13]:
# Drop columns
merged.drop(merged.columns.difference(['city','STUSAB','NAME']), 1, inplace=True)
merged

/tmp/ipykernel_137658/782222704.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  merged.drop(merged.columns.difference(['city','STUSAB','NAME']), 1, inplace=True)
/home/erichschulman/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,city,STUSAB,NAME
654324,Kennett,MO,"Kennett, MO Micro Area"
660233,Branson,MO,"Branson, MO Micro Area"
660590,Mexico,MO,"Mexico, MO Micro Area"
665306,Maryville,MO,"Maryville, MO Micro Area"
669737,West Plains,MO,"West Plains, MO Micro Area"
...,...,...,...
16356215,South Portland,ME,"Portland-South Portland, ME Metro Area"
16357576,Bangor,ME,"Bangor, ME Metro Area"
16358400,Lewiston,ME,"Lewiston-Auburn, ME Metro Area"
16359050,Portland,ME,"Portland-South Portland, ME Metro Area"


In [14]:
merged.to_csv('../data/merged.csv')

In [15]:
# Clean up entry data to merge with store sums 
entry2_data = pd.read_csv("../data/entry_loc2.csv")
lst = [' city',' village',' CDP',' town', ' borough', ' city and borough', ' municipality', ' urban county']
entry2_data['city'] = entry2_data['city'].replace(lst,'', regex=True)
entry2_data['city']= entry2_data['city'].str.strip()
remove_states = [2, 15]
entry2_data = entry2_data[~entry2_data['state'].isin(remove_states)]
entry2_data

,Unnamed: 0,Unnamed: 0.1,city,state,HD,LOW,population,income_per_capita,under44_1,under44_2,...,HD - Saint Louis,HD - Alabaster,HD - Farmington Hills,HD - Shawnee,HD - Harahan,HD - Saint Paul,HD - Draper,HD - Auburn,low warehouse distance,hd warehouse distance
0,0,0,Aullville,29,0,0,106,18648,11.0,27.0,...,301.113899,889.327730,950.448013,180.555262,1056.112267,660.656218,1564.285139,1025.937954,529.802667,180.555262
1,1,1,Gibbs,29,0,0,107,13140,14.0,6.0,...,246.722789,910.062739,801.784309,311.100623,1144.726345,541.964217,1650.414501,1037.493777,368.719657,225.133361
2,2,2,West Line,29,0,0,107,27050,17.0,5.0,...,378.075918,920.595336,1040.033968,111.347694,1044.769931,712.456889,1496.931083,1060.441735,607.141284,111.347694
3,3,3,De Witt,29,0,0,108,15269,5.0,11.0,...,270.492229,891.083783,896.972221,223.114484,1083.349860,618.343309,1595.107795,1024.862461,473.139839,223.114484
4,4,4,Arrow Point,29,0,0,111,34155,0.0,12.0,...,379.091274,721.336370,1099.007923,334.001006,798.514435,934.456092,1647.534632,864.496404,400.497175,217.382881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17308,26927,26927,South Portland,23,2,1,25515,36385,2835.0,3232.0,...,1762.824811,1844.917612,1076.570879,2187.554274,2331.107479,1824.134376,3425.143795,1806.997066,595.578960,275.787871
17309,26928,26928,Bangor,23,1,1,32098,28769,3758.0,5820.0,...,1904.862637,2013.358804,1203.945533,2316.659617,2500.105048,1914.812022,3522.370278,1978.190935,763.676891,449.640697
17310,26929,26929,Lewiston,23,0,0,36097,24541,4555.0,4515.0,...,1775.675371,1874.992742,1081.907470,2194.424656,2361.737168,1815.784456,3419.737690,1840.129976,625.311130,315.440048
17311,26930,26930,Portland,23,0,0,66735,34877,6588.0,14510.0,...,1762.152914,1845.163172,1075.534473,2186.611853,2331.391630,1822.563065,3423.697221,1807.428765,595.750431,276.553924


In [16]:
entry2_data.drop(entry2_data.columns.difference(['city','state','HD','LOW','income_per_capita','population','low warehouse distance','hd warehouse distance','STUSAB','Northeast','Midwest','South','West']), 1, inplace=True)
entry2_data

/tmp/ipykernel_137658/1306518693.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  entry2_data.drop(entry2_data.columns.difference(['city','state','HD','LOW','income_per_capita','population','low warehouse distance','hd warehouse distance','STUSAB','Northeast','Midwest','South','West']), 1, inplace=True)


,city,state,HD,LOW,population,income_per_capita,STUSAB,low warehouse distance,hd warehouse distance
0,Aullville,29,0,0,106,18648,MO,529.802667,180.555262
1,Gibbs,29,0,0,107,13140,MO,368.719657,225.133361
2,West Line,29,0,0,107,27050,MO,607.141284,111.347694
3,De Witt,29,0,0,108,15269,MO,473.139839,223.114484
4,Arrow Point,29,0,0,111,34155,MO,400.497175,217.382881
...,...,...,...,...,...,...,...,...,...
17308,South Portland,23,2,1,25515,36385,ME,595.578960,275.787871
17309,Bangor,23,1,1,32098,28769,ME,763.676891,449.640697
17310,Lewiston,23,0,0,36097,24541,ME,625.311130,315.440048
17311,Portland,23,0,0,66735,34877,ME,595.750431,276.553924


In [17]:
merged2 = entry2_data.merge(merged,how='left',on=['STUSAB','city']).drop_duplicates()
merged2

,city,state,HD,LOW,population,income_per_capita,STUSAB,low warehouse distance,hd warehouse distance,NAME
0,Aullville,29,0,0,106,18648,MO,529.802667,180.555262,NaN
1,Gibbs,29,0,0,107,13140,MO,368.719657,225.133361,NaN
2,West Line,29,0,0,107,27050,MO,607.141284,111.347694,NaN
3,De Witt,29,0,0,108,15269,MO,473.139839,223.114484,NaN
4,Arrow Point,29,0,0,111,34155,MO,400.497175,217.382881,NaN
...,...,...,...,...,...,...,...,...,...,...
17120,South Portland,23,2,1,25515,36385,ME,595.578960,275.787871,"Portland-South Portland, ME Metro Area"
17121,Bangor,23,1,1,32098,28769,ME,763.676891,449.640697,"Bangor, ME Metro Area"
17122,Lewiston,23,0,0,36097,24541,ME,625.311130,315.440048,"Lewiston-Auburn, ME Metro Area"
17123,Portland,23,0,0,66735,34877,ME,595.750431,276.553924,"Portland-South Portland, ME Metro Area"


In [18]:
# Group data and compute max
merged_grouped_max = merged2.groupby(['NAME']).max().reset_index()
merged_grouped_max

,NAME,city,state,HD,LOW,population,income_per_capita,STUSAB,low warehouse distance,hd warehouse distance
0,"Aberdeen, SD Micro Area",Aberdeen,46,0,0,28124,29901,SD,703.063678,426.919784
1,"Aberdeen, WA Micro Area",Aberdeen,53,1,0,16359,21544,WA,1566.681380,76.909798
2,"Abilene, TX Metro Area",Abilene,48,1,2,123676,23756,TX,431.553058,513.121292
3,"Ada, OK Micro Area",Ada,40,0,0,17276,22921,OK,221.739628,165.404946
4,"Adrian, MI Micro Area",Adrian,26,0,1,20640,19884,MI,100.426769,85.066260
...,...,...,...,...,...,...,...,...,...,...
891,"York-Hanover, PA Metro Area",York,42,1,1,44044,28024,PA,116.707268,84.511851
892,"Youngstown-Warren-Boardman, OH-PA Metro Area",Youngstown,42,0,1,64734,26654,PA,5282.460250,4959.536947
893,"Yuba City, CA Metro Area",Yuba City,6,1,1,66388,25088,CA,712.666555,311.363426
894,"Yuma, AZ Metro Area",Yuma,4,1,1,95573,23228,AZ,284.934227,332.850129


In [19]:
# Group data and compute sum
merged_grouped_sum = merged_grouped_max.groupby(['NAME']).sum().reset_index()
merged_grouped_sum

,NAME,state,HD,LOW,population,income_per_capita,low warehouse distance,hd warehouse distance
0,"Aberdeen, SD Micro Area",46,0,0,28124,29901,703.063678,426.919784
1,"Aberdeen, WA Micro Area",53,1,0,16359,21544,1566.681380,76.909798
2,"Abilene, TX Metro Area",48,1,2,123676,23756,431.553058,513.121292
3,"Ada, OK Micro Area",40,0,0,17276,22921,221.739628,165.404946
4,"Adrian, MI Micro Area",26,0,1,20640,19884,100.426769,85.066260
...,...,...,...,...,...,...,...,...
891,"York-Hanover, PA Metro Area",42,1,1,44044,28024,116.707268,84.511851
892,"Youngstown-Warren-Boardman, OH-PA Metro Area",42,0,1,64734,26654,5282.460250,4959.536947
893,"Yuba City, CA Metro Area",6,1,1,66388,25088,712.666555,311.363426
894,"Yuma, AZ Metro Area",4,1,1,95573,23228,284.934227,332.850129


In [20]:
merged_group_max = merged_grouped_max.drop(['HD', 'LOW', 'population'],axis=1)
merged_group_max

,NAME,city,state,income_per_capita,STUSAB,low warehouse distance,hd warehouse distance
0,"Aberdeen, SD Micro Area",Aberdeen,46,29901,SD,703.063678,426.919784
1,"Aberdeen, WA Micro Area",Aberdeen,53,21544,WA,1566.681380,76.909798
2,"Abilene, TX Metro Area",Abilene,48,23756,TX,431.553058,513.121292
3,"Ada, OK Micro Area",Ada,40,22921,OK,221.739628,165.404946
4,"Adrian, MI Micro Area",Adrian,26,19884,MI,100.426769,85.066260
...,...,...,...,...,...,...,...
891,"York-Hanover, PA Metro Area",York,42,28024,PA,116.707268,84.511851
892,"Youngstown-Warren-Boardman, OH-PA Metro Area",Youngstown,42,26654,PA,5282.460250,4959.536947
893,"Yuba City, CA Metro Area",Yuba City,6,25088,CA,712.666555,311.363426
894,"Yuma, AZ Metro Area",Yuma,4,23228,AZ,284.934227,332.850129


In [21]:
merged_group_sum = merged_grouped_sum.drop(['income_per_capita','low warehouse distance', 'hd warehouse distance'],axis=1)
merged_group_sum

,NAME,state,HD,LOW,population
0,"Aberdeen, SD Micro Area",46,0,0,28124
1,"Aberdeen, WA Micro Area",53,1,0,16359
2,"Abilene, TX Metro Area",48,1,2,123676
3,"Ada, OK Micro Area",40,0,0,17276
4,"Adrian, MI Micro Area",26,0,1,20640
...,...,...,...,...,...
891,"York-Hanover, PA Metro Area",42,1,1,44044
892,"Youngstown-Warren-Boardman, OH-PA Metro Area",42,0,1,64734
893,"Yuba City, CA Metro Area",6,1,1,66388
894,"Yuma, AZ Metro Area",4,1,1,95573


In [22]:
merged_cols = merged_grouped_max.merge(merged_grouped_sum)
merged_cols

,NAME,city,state,HD,LOW,population,income_per_capita,STUSAB,low warehouse distance,hd warehouse distance
0,"Aberdeen, SD Micro Area",Aberdeen,46,0,0,28124,29901,SD,703.063678,426.919784
1,"Aberdeen, WA Micro Area",Aberdeen,53,1,0,16359,21544,WA,1566.681380,76.909798
2,"Abilene, TX Metro Area",Abilene,48,1,2,123676,23756,TX,431.553058,513.121292
3,"Ada, OK Micro Area",Ada,40,0,0,17276,22921,OK,221.739628,165.404946
4,"Adrian, MI Micro Area",Adrian,26,0,1,20640,19884,MI,100.426769,85.066260
...,...,...,...,...,...,...,...,...,...,...
891,"York-Hanover, PA Metro Area",York,42,1,1,44044,28024,PA,116.707268,84.511851
892,"Youngstown-Warren-Boardman, OH-PA Metro Area",Youngstown,42,0,1,64734,26654,PA,5282.460250,4959.536947
893,"Yuba City, CA Metro Area",Yuba City,6,1,1,66388,25088,CA,712.666555,311.363426
894,"Yuma, AZ Metro Area",Yuma,4,1,1,95573,23228,AZ,284.934227,332.850129


In [25]:
print(merged_cols.columns)

Index(['NAME', 'city', 'state', 'HD', 'LOW', 'population', 'income_per_capita',
       'STUSAB', 'low warehouse distance', 'hd warehouse distance'],
      dtype='object')


In [ ]:
merged_cols = pd.merge(merged_cols, merged2[['NAME','Northeast','Midwest','South','West']],on='NAME')
merged_cols = merged_cols.drop(columns=['Northeast_y','Midwest_y','South_y','West_y']).drop_duplicates()
merged_cols

In [ ]:
merged_cols.to_csv("../data/entry_loc3_w_filter.csv")

In [ ]:
# Find unmerged rows
CBSA[~CBSA['NAME'].isin(merged['NAME'])].sort_values(by='NAME')['NAME'].unique()